In [ ]:
import numpy as np

# Below are some lines of IDL code to get the cross-product normal.
def Cross_product_normal_cl(E_vals,E_vec,m1,m2,mag_arr_str): 
    mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])
    m1 = pd.to_datetime(m1)
    m2 = pd.to_datetime(m2)
    
    # lv,mv,nv are in the same coordinate system as the eigenvectors
    nv = E_vec[:,0] # take B1 x B2/|B1 x B2|
    lv = E_vec[:,2]
    mv = np.cross(nv,lv)
    rat = E_vals[1]/E_vals[0]
    lxm = lv[0]
    lym = lv[1]
    lzm = lv[2]
    mxm = mv[0]
    mym = mv[1]
    mzm = mv[2]
    nxm = nv[0]
    nym = nv[1]
    nzm = nv[2]
    time = np.array(mag_arr['times'])
    bx = np.array(pd.to_numeric(mag_arr['Bx']))
    by = np.array(pd.to_numeric(mag_arr['By']))
    bz = np.array(pd.to_numeric(mag_arr['Bz']))
    # search the B-field array date column for m1
    v1 = mag_arr.date.searchsorted(m1)
    v2 = mag_arr.date.searchsorted(m2)
    
    # pick a time delta 
    dtb = pd.Timedelta(hours = 0, minutes = 0, seconds = 3)
    v1a = mag_arr.date.searchsorted(m1 - dtb)
    v2a = mag_arr.date.searchsorted(m2 + dtb)
    bv1x = bx[v1a:v1]
    bv1y = by[v1a:v1]
    bv1z = bz[v1a:v1]
    bv2x = bx[v2:v2a]
    bv2y = by[v2:v2a]
    bv2z = bz[v2:v2a]
    bv1avg = np.array([np.mean(bv1x),np.mean(bv1y),np.mean(bv1z)])
    bv2avg = np.array([np.mean(bv2x),np.mean(bv2y),np.mean(bv2z)])
    bmag1 = np.sqrt(np.sum(np.dot(bv1avg,bv1avg)))
    # original bmag2 = np.sqrt(np.sum(np.multiply(bv2avg,bv2avg)))
    bmag2 = np.sqrt(np.sum(np.dot(bv2avg,bv2avg)))
    normal = np.cross(bv1avg,bv2avg)
    normag = np.sqrt(np.sum(np.dot(normal,normal)))
    nc = normal/normag  # N-direction
    # #Angle between cross-product & MVAB normals
    #n12ang = np.arccos(np.sum(nv*nc))*180./!dpi
    
    # not sure if we need this part
#     n12ang = np.arccos(np.sum(nv*nc))*180
#     if n12ang >= 90.0:
#         nc = -nc

# TEST 1
# flip $N_{GSE}$ to $-N_{GSE}$ if needed for all events with $N_{y}>0$
    if nym > 0:
        nc =-nc

#    #TEST 2
# # flip $N_{GSE}$ to $-N_{GSE}$ if needed for all events with $N_{y}>0$
#     if nc[1] >0:
#         nyc = -nc[1]
#     else:
#         nyc=nc[1]
        
    mvec = np.cross(nc,E_vec[:,2])
    mmag = np.sqrt(np.sum(np.dot(mvec,mvec)))
    mc = mvec/mmag    # M-direction
    lc = np.cross(mc,nc)  # L-direction
    # angle = np.arccos(np.sum(np.multiply(E_vec[:,2],lc)))*180
    # Angle between cross-product & MVAB normals
    n12ang = np.arccos(np.sum(np.dot(nv,nc)))*180
    lxc = lc[0]
    lyc = lc[1]
    lzc = lc[2]
    mxc = mc[0]
    myc = mc[1]
    mzc = mc[2]
    nxc = nc[0]
    nyc = nc[1]
    nzc = nc[2]
    nang = n12ang

    return lc,mc,nc,lv,mv,nv